# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value

    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

sum = 0
for person in people_1():
    sum = sum + person['Age']
print(sum)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# for person in people_2():
#     print(person)


140


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [6]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

## 2. Append generator to table

In [9]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace")

info = generators_pipeline.run(people_2(),
										table_name="people_1",
										write_disposition="append")

In [14]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

rides = conn.sql("SELECT * FROM people_1").df()
display(rides)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708905559.5600934,8YJCrjjtj6CKRg,None
1,2,Person_2,27,City_A,1708905559.5600934,wXKsV85QflZmIg,None
2,3,Person_3,28,City_A,1708905559.5600934,mSiqeq3r6bRsjA,None
3,4,Person_4,29,City_A,1708905559.5600934,bTtH6M8Y02yRFg,None
4,5,Person_5,30,City_A,1708905559.5600934,VZHIufBW97SaIA,None
5,3,Person_3,33,City_B,1708905560.2759938,9bYsyQ9KnJY0lA,Job_3
6,4,Person_4,34,City_B,1708905560.2759938,j3IzMDqyQ8Vhwg,Job_4
7,5,Person_5,35,City_B,1708905560.2759938,QXP/GZa8CVml3g,Job_5
8,6,Person_6,36,City_B,1708905560.2759938,VoLtmCkq9D0kiw,Job_6
9,7,Person_7,37,City_B,1708905560.2759938,PSVzuyhMwnwd/w,Job_7


## 3. Merge generator

In [16]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators_new')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_1",
										write_disposition="replace",
                    primary_key="ID")

info = generators_pipeline.run(people_2(),
										table_name="people_1",
										write_disposition="merge",
                    primary_key="ID")

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT * FROM people_1").df()
display(people)

sum_of_ages = conn.sql("SELECT sum(age) FROM people_1").df()
display(sum_of_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708906135.2153134,uuMMuFY4LEZGxw,None
1,2,Person_2,27,City_A,1708906135.2153134,lJy58ksRi3kAzw,None
2,8,Person_8,38,City_B,1708906135.8710907,w6dyDZ/W/xTTTQ,Job_8
3,7,Person_7,37,City_B,1708906135.8710907,izxxkqiI89h/rQ,Job_7
4,4,Person_4,34,City_B,1708906135.8710907,qBjYekuZXEQuUw,Job_4
5,5,Person_5,35,City_B,1708906135.8710907,ECG6ftKbEu7b1w,Job_5
6,3,Person_3,33,City_B,1708906135.8710907,v/mW/7hdf2MJmw,Job_3
7,6,Person_6,36,City_B,1708906135.8710907,bUmt3rPOyk/5Ng,Job_6


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX